In [1]:
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import Dataset, DataLoader
import os

from pytorch_lightning import LightningModule, Trainer, LightningDataModule, Callback
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger

import pandas as pd
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
from src.etci_dataset import ETCIDataset, ETCIDataModule
from src.model import SegModule
from src.utils import *
import segmentation_models_pytorch as smp
import wandb
!pwd
os.chdir('src')
!pwd

/home/genrev_kiel_hizon/SAR_flood_segmentation
/home/genrev_kiel_hizon/SAR_flood_segmentation/src


In [2]:
model = smp.Unet(
        encoder_name='efficientnet-b7',
        encoder_weights='imagenet',
        in_channels=3,
        classes=1
    )

In [3]:
wandb_logger = WandbLogger(project='sar_seg', log_model='all')
model_checkpoint = ModelCheckpoint(
        dirpath=os.path.join('SAR_flood_segmentation', "checkpoints"),
        filename="sar-best-miou",
        save_top_k=1,
        verbose=True,
        monitor='val_miou',
        mode='max',
    )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: khizon. Use `wandb login --relogin` to force relogin


In [4]:
early_stop_callback = EarlyStopping(monitor="val_miou", min_delta=0.25, patience=5, verbose=False, mode="max")

In [5]:
trainer = Trainer(max_epochs=1,
                      logger=wandb_logger,
                      gradient_clip_val=0.5,
                      callbacks=[model_checkpoint, early_stop_callback])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
datamodule=ETCIDataModule('../data/', batch_size=8, num_workers=1)
datamodule.setup()

In [7]:
unet_model = SegModule(model)

In [8]:
trainer.fit(unet_model, datamodule=datamodule)
wandb.finish()


  | Name   | Type               | Params
----------------------------------------------
0 | model  | Unet               | 67.1 M
1 | loss   | BCEWithLogitsLoss  | 0     
2 | metric | BinaryJaccardIndex | 0     
----------------------------------------------
67.1 M    Trainable params
0         Non-trainable params
67.1 M    Total params
268.381   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]